In [34]:
!pip install yfinance pandas numpy matplotlib mplfinance


[notice] A new release of pip is available: 23.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime
import os

In [7]:
def fetch_historical_data(ticker, start_date, end_date, type='day'):
    # Download data using yfinance
    data_daily = yf.download(ticker, start=start_date, end=end_date, interval='1d')
    data_weekly = yf.download(ticker, start=start_date, end=end_date, interval='1wk')
    data_monthly = yf.download(ticker, start=start_date, end=end_date, interval='1mo')
    if type == 'day':
        return data_daily
    elif type == 'week':
        return data_weekly
    elif type == 'month':
        return data_monthly
    
start_date = '2013-03-01'
    
tsla_data = fetch_historical_data('TSLA', start_date, datetime.today().strftime('%Y-%m-%d'))
tsla_data.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-03-01,2.333333,2.338667,2.283333,2.310000,2.310000,23199000
2013-03-04,2.318000,2.388667,2.313333,2.372000,2.372000,26365500
2013-03-05,2.400000,2.461333,2.386000,2.443333,2.443333,31305000
2013-03-06,2.467333,2.525333,2.464667,2.512667,2.512667,17250000
2013-03-07,2.515333,2.576667,2.458667,2.548667,2.548667,17374500


In [18]:
def calc_moving_av(data):
    data_out = data.copy()
    data_out['MA_5'] = data_out['Close'].rolling(window=5).mean()
    data_out['MA_10'] = data_out['Close'].rolling(window=10).mean()
    data_out['MA_20'] = data_out['Close'].rolling(window=20).mean()
    data_out['MA_60'] = data_out['Close'].rolling(window=60).mean()
    data_out['MA_120'] = data_out['Close'].rolling(window=120).mean()
    data_out['MA_240'] = data_out['Close'].rolling(window=240).mean()
    return data_out

tsla_ma = calc_moving_av(tsla_data)
tsla_ma.head()


,Open,High,Low,Close,Adj Close,Volume,MA_5,MA_10,MA_20,MA_60,MA_120,MA_240
Date,,,,,,,,,,,,
2013-03-01,2.333333,2.338667,2.283333,2.310000,2.310000,23199000,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-04,2.318000,2.388667,2.313333,2.372000,2.372000,26365500,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-05,2.400000,2.461333,2.386000,2.443333,2.443333,31305000,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-06,2.467333,2.525333,2.464667,2.512667,2.512667,17250000,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-07,2.515333,2.576667,2.458667,2.548667,2.548667,17374500,2.437333,NaN,NaN,NaN,NaN,NaN


In [22]:
def stoch(data, lookback_period=20, smooth_k=10, smooth_d=10):
    # Calculate raw %K
    low_min = data['Low'].rolling(window=lookback_period).min()
    high_max = data['High'].rolling(window=lookback_period).max()
    raw_k = 100*((data['Close'] - low_min) / (high_max - low_min))
    
    #Smooth %K
    smoothed_k = raw_k.rolling(window=smooth_k).mean()
    
    #Calculate %D as the moving average of %K
    smoothed_d = smoothed_k.rolling(window=smooth_d).mean()
    
    return smoothed_k, smoothed_d

k_20, d_20 = stoch(tsla_ma)
k_10, d_10 = stoch(tsla_ma, 10, 5, 5)
k_5, d_5 = stoch(tsla_ma, 5, 3, 3)

tsla_ma_stoch = tsla_ma.copy()
tsla_ma_stoch['Sto_K20_10'] = k_20
tsla_ma_stoch['Sto_D20_10'] = d_20
tsla_ma_stoch['Sto_K10_5'] = k_10
tsla_ma_stoch['Sto_D10_5'] = d_10
tsla_ma_stoch['Sto_K5_3'] = k_5
tsla_ma_stoch['Sto_D5_3'] = d_5

tsla_ma_stoch

,Open,High,Low,Close,Adj Close,Volume,MA_5,MA_10,MA_20,MA_60,MA_120,MA_240,Sto_K20_10,Sto_D20_10,Sto_K10_5,Sto_D10_5,Sto_K5_3,Sto_D5_3
Date,,,,,,,,,,,,,,,,,,
2013-03-01,2.333333,2.338667,2.283333,2.310000,2.310000,23199000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-04,2.318000,2.388667,2.313333,2.372000,2.372000,26365500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-05,2.400000,2.461333,2.386000,2.443333,2.443333,31305000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-06,2.467333,2.525333,2.464667,2.512667,2.512667,17250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-07,2.515333,2.576667,2.458667,2.548667,2.548667,17374500,2.437333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-10,220.070007,226.399994,218.639999,226.169998,226.169998,78891100,220.550000,214.870001,214.972502,218.029167,195.341334,207.679458,60.820388,60.120215,60.181817,50.101741,34.438822,45.369574
2024-09-11,224.550003,228.470001,216.800003,228.130005,228.130005,83548600,222.294000,216.762001,215.987502,218.864501,195.778584,207.627917,63.760067,60.881886,60.796056,55.114632,54.011926,43.120698
2024-09-12,224.660004,231.449997,223.830002,229.809998,229.809998,72020000,222.222000,219.168001,217.409002,219.570668,196.253500,207.558875,67.396326,61.788863,60.201005,57.097799,73.159389,53.870046


In [25]:
def RSI(data, period=14):
    delta = data['Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

tsla_ma_stoch_rsi = tsla_ma_stoch.copy()
tsla_ma_stoch_rsi['RSI'] = RSI(tsla_data)
tsla_ma_stoch_rsi = tsla_ma_stoch_rsi.iloc[240:]
tsla_ma_stoch_rsi

,Open,High,Low,Close,Adj Close,Volume,MA_5,MA_10,MA_20,MA_60,MA_120,MA_240,Sto_K20_10,Sto_D20_10,Sto_K10_5,Sto_D10_5,Sto_K5_3,Sto_D5_3,RSI
Date,,,,,,,,,,,,,,,,,,,
2014-02-12,13.052000,13.218000,12.954667,13.021333,13.021333,77605500,12.712133,12.319466,11.958667,10.236711,10.757689,8.265083,88.473501,88.015652,83.331831,72.469908,82.532449,85.039024,60.381600
2014-02-13,12.889333,13.514667,12.883333,13.308667,13.308667,120439500,12.995467,12.431400,12.077000,10.308022,10.781317,8.310653,88.110380,87.770228,88.637605,77.703301,81.127992,84.917682,69.563856
2014-02-14,13.206667,13.458667,13.133333,13.215333,13.215333,92370000,13.151467,12.543533,12.167867,10.393189,10.801533,8.355536,87.859578,87.544756,86.037004,81.572446,75.622652,79.761031,73.687691
2014-02-18,13.682667,13.733333,13.424000,13.580000,13.580000,139992000,13.246667,12.720800,12.280167,10.479422,10.823467,8.401650,89.104693,87.536684,86.611878,84.252364,78.611438,78.454027,72.171621
2014-02-19,13.580000,13.580000,12.894000,12.909333,12.909333,242535000,13.206933,12.820200,12.336700,10.560011,10.838261,8.444819,87.584982,87.508999,83.263449,85.576353,50.752718,68.328936,64.380560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-10,220.070007,226.399994,218.639999,226.169998,226.169998,78891100,220.550000,214.870001,214.972502,218.029167,195.341334,207.679458,60.820388,60.120215,60.181817,50.101741,34.438822,45.369574,52.406718
2024-09-11,224.550003,228.470001,216.800003,228.130005,228.130005,83548600,222.294000,216.762001,215.987502,218.864501,195.778584,207.627917,63.760067,60.881886,60.796056,55.114632,54.011926,43.120698,52.311644
2024-09-12,224.660004,231.449997,223.830002,229.809998,229.809998,72020000,222.222000,219.168001,217.409002,219.570668,196.253500,207.558875,67.396326,61.788863,60.201005,57.097799,73.159389,53.870046,60.165619


In [33]:
def market_tendency(data, periods=5):
    future_returns = data['MA_5'].pct_change(periods=periods)
    return future_returns

tsla_ma_stoch_rsi['Market_Tendency']=market_tendency(tsla_ma)
tsla_ma_stoch_rsi['Market_Tendency']=tsla_ma_stoch_rsi['Market_Tendency'].shift(-5)
tsla_ma_stoch_rsi = tsla_ma_stoch_rsi.iloc[:-5]
tsla_ma_stoch_rsi
    

C:\Users\hummi\AppData\Local\Temp\ipykernel_11756\1048772763.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tsla_ma_stoch_rsi['Market_Tendency']=market_tendency(tsla_ma)
C:\Users\hummi\AppData\Local\Temp\ipykernel_11756\1048772763.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tsla_ma_stoch_rsi['Market_Tendency']=tsla_ma_stoch_rsi['Market_Tendency'].shift(-5)


,Open,High,Low,Close,Adj Close,Volume,MA_5,MA_10,MA_20,MA_60,MA_120,MA_240,Sto_K20_10,Sto_D20_10,Sto_K10_5,Sto_D10_5,Sto_K5_3,Sto_D5_3,RSI,Market_Tendency
Date,,,,,,,,,,,,,,,,,,,,
2014-02-12,13.052000,13.218000,12.954667,13.021333,13.021333,77605500,12.712133,12.319466,11.958667,10.236711,10.757689,8.265083,88.473501,88.015652,83.331831,72.469908,82.532449,85.039024,60.381600,0.054289
2014-02-13,12.889333,13.514667,12.883333,13.308667,13.308667,120439500,12.995467,12.431400,12.077000,10.308022,10.781317,8.310653,88.110380,87.770228,88.637605,77.703301,81.127992,84.917682,69.563856,0.041532
2014-02-14,13.206667,13.458667,13.133333,13.215333,13.215333,92370000,13.151467,12.543533,12.167867,10.393189,10.801533,8.355536,87.859578,87.544756,86.037004,81.572446,75.622652,79.761031,73.687691,0.048867
2014-02-18,13.682667,13.733333,13.424000,13.580000,13.580000,139992000,13.246667,12.720800,12.280167,10.479422,10.823467,8.401650,89.104693,87.536684,86.611878,84.252364,78.611438,78.454027,72.171621,0.085918
2014-02-19,13.580000,13.580000,12.894000,12.909333,12.909333,242535000,13.206933,12.820200,12.336700,10.560011,10.838261,8.444819,87.584982,87.508999,83.263449,85.576353,50.752718,68.328936,64.380560,0.149114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,218.750000,219.089996,211.009995,213.210007,213.210007,59301200,217.712006,215.075003,210.879002,211.613667,191.823084,209.592333,54.356004,34.159991,68.576320,79.631382,24.636872,40.789731,56.911915,-0.039143
2024-08-27,213.250000,215.660004,206.940002,209.210007,209.210007,62821400,215.334006,215.213004,210.208502,212.132500,192.095334,209.333625,56.143359,37.926844,59.550950,75.417967,28.240395,32.804814,60.139462,-0.019059
2024-08-28,209.720001,211.839996,202.589996,205.750000,205.750000,64116400,211.830005,215.650003,208.892502,212.623501,192.321167,209.040750,58.326556,41.962501,44.753129,66.346692,14.361027,22.412765,54.191438,0.020224


In [47]:
tsla_ma_stoch_rsi.index[0].strftime('%Y-%m-%d')

'2014-02-12'

In [58]:
output_dir = 'tsla_candlestick_charts'
os.makedirs(output_dir, exist_ok=True)

for i in range(len(tsla_ma_stoch_rsi)):
    data_wind = tsla_ma_stoch_rsi[i:i+100]
    addplots = [
        mpf.make_addplot(data_wind['MA_5'], color='red'),
        mpf.make_addplot(data_wind['MA_10'], color='orange'),
        mpf.make_addplot(data_wind['MA_20'], color='yellow'),
        mpf.make_addplot(data_wind['MA_60'], color='green'),
        mpf.make_addplot(data_wind['MA_120'], color='blue'),
        mpf.make_addplot(data_wind['MA_240'], color='purple'),
        # add stoch
        mpf.make_addplot(data_wind['Sto_K20_10'], panel=2, color='blue'),
        mpf.make_addplot(data_wind['Sto_D20_10'], panel=2, color='red'),
        mpf.make_addplot(data_wind['Sto_K10_5'], panel=3, color='blue'),
        mpf.make_addplot(data_wind['Sto_D10_5'], panel=3, color='red'),
        mpf.make_addplot(data_wind['Sto_K5_3'], panel=4, color='blue'),
        mpf.make_addplot(data_wind['Sto_D5_3'], panel=4, color='red'),
        # add rsi
        mpf.make_addplot(data_wind['RSI'], panel=5, color='green'),
        # add market tendency
        mpf.make_addplot(data_wind['Market_Tendency'], panel=6, color='black')        
    ]
    
    file_path = os.path.join(output_dir, f'tsla_chart_{data_wind.index[-1].strftime("%Y-%m-%d")}.png')
    
    style = mpf.make_mpf_style(
        base_mpf_style='classic',
        y_on_right=False,
        rc={'axes.spines.left': False, 'axes.spines.right': False, 'axes.labelcolor': 'none', 'xtick.color': 'none', 'ytick.color': 'none'}
    )
    
    mpf.plot(data_wind, type='candle', volume=True, addplot=addplots, panel_ratios=(5,1,1,1,1,1,1), figscale=2, savefig=file_path, style=style,
             hlines=[dict(hlines=[0, 20, 80], colors=['gray'],linewidths=0.5, linestyle='--',panels=[2,3,4,5]),
                     dict(hlines=[0], colors=['black'],linewidths=0.5, linestyle='--', panels=[6])])

TypeError: kwarg "hlines" validator returned False for value: "[{'hlines': [0, 20, 80], 'colors': ['gray'], 'linewidths': 0.5, 'linestyle': '--', 'panels': [2, 3, 4, 5]}, {'hlines': [0], 'colors': ['black'], 'linewidths': 0.5, 'linestyle': '--', 'panels': [6]}]"
    'Validator'   : lambda value: _hlines_validator(value) },